# Ca II 8542 NICOLE inversion - full frame

In [337]:
from __future__ import print_function
%matplotlib tk
from ReductionImports import *
import  NicoleNative as nic
from numpy import copy as copy
import struct
setrcParams()
# dir_init = getFileFilesDir(2)
dir_init = '/mnt/Data/CaII Polarimeter v2/Level-2/'
dir_nicole = '/home/hp/Dropbox/Inversions/NICOLE.git/run/'
print(dir_init)

/mnt/Data/CaII Polarimeter v2/Level-2/


## NICOLE specific routines

In [446]:
# Read ASCII model file
def read_ascii_model(MODFILE):
    F = open(MODFILE)
    LIN8 = loadtxt(MODFILE, skiprows=2)
    HEAD = F.readline()
    PT2 = F.readline()
    PAR10 = [r'Macro turbulent velocity', r'Stray light fraction', 
             r'$log(\tau_{5000})$', r'T in $K$', r'Electron Presuure in $dyne/cm^2$', r'Microtubulence in $cm/s$', 
             r'$B_z$ in $Gauss$', r'$v_z$ in $cm/s$', r'$B_x$ in $Gauss$', r'$B_y$ in $Gauss$']
    return PAR10, HEAD, PT2, LIN8
# Create ASCII model file
def create_ascii_model(MODFILE, HEAD, PT2, LIN8):
    F = open(MODFILE, 'w')
    F.write(''.join([HEAD, PT2]))
    savetxt(F, LIN8, '%0.6f')
    F.close()
# Read NICOLE native format model file
def read_native_model(MODFILE):
    SIZE = os.path.getsize(MODFILE)
    F = open(MODFILE, 'rb')
    HEAD = F.read(16)
    NX = struct.unpack('@I', F.read(4))[0]
    NY = struct.unpack('@I', F.read(4))[0]
    NZ = struct.unpack('@Q', F.read(8))[0]
    SIG = 22*NZ+11+92
    F.seek(0)
    F.seek(SIG*8)
    S = []
    for i in range(NX*NY*SIG):
        S.append(struct.unpack('@d', F.read(8))[0])
    MOD = reshape(array(S, dtype=float64), (SIG, NY, NX), 'F')
    LIN22 = reshape(MOD[0:22*NZ,:,:], (NZ, 22, NY, NX), 'F')
    PT11 = MOD[22*NZ:22*NZ+11]
    ABU = MOD[22*NZ+11:22*NZ+11+92]
    PAR34 = [r'$z$', r'$log(\tau)$', r'T', r'$P_{gas}$', r'$\rho$', r'$P_{el}$', 
             r'$v_z$', r'$v_{mic}$', r'$B_z$', r'$B_x$', r'$B_y$', 
             r'$B_{z (local)}$', r'$B_{y (local)}$', r'$B_{x (local)}$', 
             r'$v_{z (local)}$', r'$v_{y (local)}$', r'$v_{x (local)}$',
             r'$nH$', r'$nH^-$', r'$nH^+$', r'$nH_2$', r'$nH_2^+$']
    try:
        MODERRFILE = MODFILE + '.err'
        F = open(MODERRFILE, 'rb')
        S = []
        for i in range(int(SIZE/8-SIG)):
            S.append(struct.unpack('@d', F.read(8))[0])
        MODERR = reshape(array(S, dtype=float64), (SIG, NY, NX), 'F')
        MERR = MODERR[:,0,0]
        LIN22E = reshape(MERR[0:22*NZ], (NZ, 22), 'F')
        PT11E = MERR[22*NZ:22*NZ+11]
        ABUE = MERR[22*NZ+11:22*NZ+11+92]
    except: 
        LIN22E = 0*np.copy(LIN22)
    return PAR34, HEAD, [LIN22, LIN22E], PT11, ABU
# Read NICOLE native profile file
def read_native_profile(PROFILES):
    STOKES = [0]*len(PROFILES)
    for p, PROFILE in enumerate(PROFILES):
        SIZE = os.path.getsize(PROFILE)
        F = open(PROFILE, 'rb')
        SIG = F.read(16)
        print(PROFILE, '\n', SIG)
        NX = struct.unpack('@I', F.read(4))[0]
        NY = struct.unpack('@I', F.read(4))[0]
        NW = struct.unpack('@Q', F.read(8))[0]
        print(NX, NY, NW)
        F.seek(0)
        F.seek(4*NW*8)
        S = []
        for i in range(int(SIZE/8-4*NW)):
            S.append(struct.unpack('@d', F.read(8))[0])
        STOKES[p] = reshape(array(S, dtype=float64), (4, NW, NX, NY), 'F')
    return STOKES
# Set NICOLE input parameters for NICOLE.input
def set_nicole_input_params(**kwargs):
    TEXT = ''
    for key, value in kwargs.items():
        STR = key + '=' + str(value) + '\n'
        TEXT += STR.replace('__', ' ')
    return TEXT
# Set region and line information for NICOLE.input
def set_nicole_regions_lines(TEXT, HEAD, **kwargs):
    TEXT += '[' + HEAD + ']\n'
    for key, value in kwargs.items():
        STR = '\t' + key + '=' + str(value) + '\n'
        TEXT += STR.replace('__', ' ')
    return TEXT
# Plot Stokes profiles
def plot_profile(FIG, STOKES, **kwargs):
    TIT = ['$I/I_0$', '$Q/I$', '$U/I$', '$V/I$']
    for S in STOKES:
        for i in range(4):
            AX = FIG.add_subplot(221+i, title=TIT[i])
            if(i != 0):
                TEMP = (S[i]/S[0]).ravel()
                AX.axhline(0, color='k', ls='--', lw=0.5)
            else:
                TEMP = S[0].ravel()
                AX.axhline(1, color='k', ls='--', lw=0.5)
#             AX.axvline(pix_caii,color='k', ls='--', lw=0.5)
            AX.plot(TEMP, **kwargs)       
# Plot important model parameters
def plot_model_bvtp(FIG, MODELS, LOGTAU, **kwargs):
    TIT = ['$B_{los}$', '$B_x$', '$B_y$', '$v_{los}$', '$T$', '$v_{mic}$']
    AX = [FIG.add_subplot(231+i, title=TIT[i]) for i in range(len(TIT))]
    for M in MODELS:
        if (8 in M.shape):
            for i,j in enumerate([4,6,7,5,1,3]):
                AX[i].plot(M[:,0], M[:,j],**kwargs)
        else:
            if (LOGTAU): XAX = 1
            else: XAX = 0    
            for i,j in enumerate([8,9,10,6,2,7]):
                AX[i].plot(M[:,XAX], M[:,j], **kwargs)

## Load Stokes data file

In [3]:
stokes_file = getFileFilesDir(0, initialdir = dir_init, 
                                 filetypes  = (('FITS file', '*STOKES*.fits'),), 
                                 title = 'Choose the stack of Stokes files...') # Select the file
hdu = pf.open(stokes_file) # Open with fits package
stokes_obs = float32(hdu[0].data) # Read the data
header = hdu[0].header # Read the header
hdu.close() # Clos HDU
print(stokes_file, ' is loaded')
path_dirs = os.path.normpath(stokes_file).split(SEP)
dir_save = 'ZIMPOL3'
# dir_save = SEP.join(path_dirs[0:-2]) + SEP + 'Level-3' + SEP + dir_save + SEP
dir_save = dir_nicole + dir_save + SEP
print(dir_save)
if not os.path.exists(dir_save):
    os.makedirs(dir_save)

/mnt/Data/CaII Polarimeter v2/Level-2/STOKES_ZIMPOL3.fits  is loaded
/home/hp/Dropbox/Inversions/NICOLE.git/run/ZIMPOL3/


In [72]:
catlinefile = getFileFilesDir(0, initialdir = '.\\',
                               filetypes  = (('Text file', '*8542*.txt'),),
                               title = 'Select line profile from flat...') # Select the file
catline = loadtxt(catlinefile)

## Parameters

In [4]:
line = mean(stokes_obs[0,:,:,:,],axis=(1,2))
plot(line)

In [18]:
# Si I
lin_cen = 158
lin_wid = 20
xx = arange(lin_cen-lin_wid//2, lin_cen+lin_wid//2)
yy = line[lin_cen-lin_wid//2:lin_cen+lin_wid//2]
mod_sii = models.Const1D() - models.Gaussian1D(amplitude=yy.max()-yy.min(), mean=lin_cen, stddev=8)
Fitter = fitting.LevMarLSQFitter()
Fit = Fitter(mod_sii, xx, yy)
previewPlot(xx, yy, xx, Fit(xx))
pix_sii = Fit.mean_1.value
print(pix_sii)

156.16940109991285


In [17]:
# Fe I
lin_cen = 294
lin_wid = 12
xx = arange(lin_cen-lin_wid//2, lin_cen+lin_wid//2)
yy = line[lin_cen-lin_wid//2:lin_cen+lin_wid//2]
mod_fei = models.Const1D() - models.Gaussian1D(amplitude=yy.max()-yy.min(), mean=lin_cen, stddev=3)
Fitter = fitting.LevMarLSQFitter()
Fit = Fitter(mod_fei, xx, yy)
previewPlot(xx, yy, xx, Fit(xx))
pix_fei = Fit.mean_1.value
print(pix_fei)

293.0056949928136


In [146]:
ns, nw, ny, nx = stokes_obs.shape
wav_caii = 8542.089
wav_atm1 = 8540.8
wav_fei = 8538.015
wav_sii = 8536.165
plate_scale = 0.01
lin_disp = (wav_fei-wav_sii)/(pix_fei-pix_sii)
pix_caii = pix_sii+(wav_caii-wav_sii)/lin_disp
nw_ran = (arange(nw) - pix_caii)*lin_disp + wav_caii
stokes = np.copy(stokes_obs)
stokes[1::] /= stokes[0]

In [163]:
catwav = catline[300:-650,0]
catint = catline[300:-650,1]/10000.0
catint = gaussian_filter1d(catint, 4)
stokes_comp = mean(stokes, axis=(2,3))
stokes_comp[0] /= max(stokes_comp[0])*1.1
stokes_comp[1::] *= stokes_comp[0]
previewPlot(catwav, catint, 'k--', nw_ran, stokes_comp[0,:], 'k-')

## Select RoI for profiles

In [394]:
stokes_crop = delete(stokes_obs[:,:,20:-20,:], [0,1,2,3,4,9,18,19,23,24,25,26,27], axis=3)

In [402]:
x_bin, y_bin = 15, 100
x_beg, x_end, y_beg, y_end = get_xy(stokes_crop[0,700], x_bin, y_bin)
x_beg, y_beg = 0, 0
x_end, y_end = x_beg+x_bin, y_beg+y_bin
print(x_beg, y_beg, x_bin, y_bin)
# Relevant dir and names
suffix = '_'.join(['CaII', str(x_beg), str(y_beg), str(x_bin), str(y_bin)])
print(suffix)
model_fit = dir_save + 'modfit_' + suffix + '.mod'
model_fit2 = dir_save + 'modfit2_' + suffix + '.mod'
model_fit3 = dir_save + 'modfit3_' + suffix + '.mod'
profile_obs = dir_save + 'proobs_' + suffix + '.pro'
profile_fit = dir_save + 'profit_' + suffix + '.pro'
profile_fit2 = dir_save + 'profit2_' + suffix + '.pro'
profile_fit3= dir_save + 'profit3_' + suffix + '.pro'
stokes_disp = mean(stokes_obs[:,:, y_beg:y_end, x_beg:x_end], axis=(2,3))
plot_profile(figure(0), [stokes_disp])

0 0 15 100
CaII_0_0_15_100


## Tweak observed profiles

In [429]:
win_smooth = 3
wav_off = 0
wav_n = 200
pix_start = int(pix_caii-wav_n/2)
# wav_n = 600
# pix_start = 240
wav_ran = reshape(nw_ran[pix_start:pix_start+wav_n], (1,wav_n)) + wav_off
stokes_temp = np.copy(stokes_obs[:,pix_start:pix_start+wav_n, y_beg:y_end, x_beg:x_end])
stokes_temp[1::] /= stokes_temp[0] 
stokes_temp[0] /= np.max(stokes_temp, axis=1)[0]*1.1
stokes_temp[1::] *= stokes_temp[0]
stokes_temp = 0.5*(stokes_temp[:,:,0::2,:]+stokes_temp[:,:,1::2,:])
plot_profile(figure(0), [stokes_temp[:,:,6,0]])
# plot_profile(figure(0), [stokes_temp])
# previewImage(stokes_temp[0,0,:,:])

In [430]:
stokes_temp.shape

(4, 200, 50, 15)

In [404]:
i_noise = std(gaussian_filter1d(stokes_temp[0], 3, axis=0)-stokes_temp[0])
q_noise = std(gaussian_filter1d(stokes[1], 3, axis=0)-stokes[1])
u_noise = std(gaussian_filter1d(stokes[2], 3, axis=0)-stokes[2])
v_noise = std(gaussian_filter1d(stokes[3], 3, axis=0)-stokes[3])
print(i_noise, q_noise, u_noise, v_noise)

0.008297005 0.0009698602 0.0022969719 0.0011191663


## Create NICOLE native profile

In [406]:
import struct
try: 
    delete(profile_obs)
except:
    pass
f = open(profile_obs, 'wb')
prohead =  b'nicole2.3bp     '
f.write(prohead)
s, nw0, ny0, nx0 = stokes_temp.shape
f.write(struct.pack('@I', ny0))
f.write(struct.pack('@I', nx0))
f.write(struct.pack('@Q', nw0))
for i in range(4*wav_n-4):
    f.write(struct.pack('@Q', 0))
for i in range(nx0):
    for j in range(ny0):
        for k in range(nw0):
            for s in [0,1,2,3]:
                f.write(struct.pack('@d', stokes_temp[s,k,j,i]))
f.close()

In [407]:
weights = 10000+zeros([wav_n,5])
weights[:,0] = wav_ran
weights[80:140, 1] = i_noise
weights[:, 2] = q_noise
weights[:, 3] = u_noise
weights[:,4] = v_noise
savetxt('../Weights.pro', weights, fmt='{:^10}'.format('%s'), delimiter=' ')

## Guess model

In [409]:
# New model
model_guess = dir_save + 'modguess_' + suffix + '.model'
model_default = 'falc.model'
me_pars, me_head, me_const, me_out = nic.read_ascii_model(dir_nicole + model_default)
me_const = ' ' + str(37170*3) + ' 0.0\r\n'
# me_out = rebin(me_out, [50,8])
N = me_out.shape[0]
tau = np.copy(me_out[:,0])
tau -= tau.min() - 0.5
tau /= tau.max()
tau /= tau
# me_out[:,5] = 200000*tau  # v_los
# me_out[:,4] = -400 # b_los
# me_out[:,6] = 2*bmap[1,y_beg,x_beg]  # b_x
# me_out[:,7] = -2*bmap[1,y_beg,x_beg] # b_y
nic.create_ascii_model(model_guess, me_head, me_const, me_out)
mi_pars, mi_head, mi_const, mi_plot = nic.read_ascii_model(model_guess)
# fig = figure(None, figsize=(10,6))
# nic.plot_model_bvtp(fig, [mi_plot])

## Invert

In [411]:
nicole_input = dir_nicole + 'NICOLE.input'
INPUT = nic.set_nicole_input_params(Mode = 'I',
                                    Printout__detail = 1,
                                    Input__model = model_guess,
                                    Observed__profiles = profile_obs,
                                    Output__profiles = profile_fit,
                                    Heliocentric__angle = 0.96, 
                                    Output__model = model_fit,
                                    noise__level = 0.001,
                                    Maximum__number__of__inversions=5,
                                    Continuum__reference = 3,
                                    Debug__mode = 1)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Region 1',
                                     First__wavelength = wav_ran[0,0], 
                                     Wavelength__step = lin_disp,
                                     Number__of__wavelengths = wav_n)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Line 1',
                                     Line = 'CaII__8542',
                                     Mode = 'NLTE')
# INPUT = nic.set_nicole_regions_lines(INPUT, 'Line 2',
#                                      Line = 'FeI__8538',
#                                      Mode = 'LTE')
INPUT = nic.set_nicole_regions_lines(INPUT, 'Nodes',
                                     Temperature = 6,
                                     Velocity = 2,
                                     Microturbulence = 1,
                                     Bz = 2,
                                     By = 2,
                                     Bx = 2)
# INPUT = nic.set_nicole_regions_lines(INPUT, 'NLTE',
#                                      Linear__formal__solution = 1)

F = open(nicole_input, 'w')
F.write(INPUT)
F.close()

## Plot model, profiles

In [459]:
ix, ix2, iy, iy2 = get_xy(mean(stokes_temp[0], axis=0), 1,1)
s_out = read_native_profile([profile_fit])[0]
# s_out2 = [gaussian_filter1d(s_out[0], 3, axis=1)]
fig = figure(None, figsize=(10,6))
nic.plot_profile(fig, [stokes_temp[:,:,iy, ix]], color='k', ls='-.', lw=0.5)
nic.plot_profile(fig, [s_out[:,:,iy,ix]], color='k', ls='--', lw=1)
fig.tight_layout()
savefig(dir_save+'profit_'+suffix+'.eps', format='eps', dpi=1200)

/home/hp/Dropbox/Inversions/NICOLE.git/run/ZIMPOL3/profit_CaII_0_0_15_100.pro 
 b'nicole2.3bp     '
50 15 200


/home/hp/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [458]:
mo_pars, mo_head, mo_plot, mo_const, mo_abu = read_native_model(model_fit)
fig = figure(None, figsize=(10,6))
nic.plot_model_bvtp(fig, [mi_plot], True, color='k', ls=':')
nic.plot_model_bvtp(fig, [mo_plot[0][:,:,0,IND]], True, color='k', ls = '--')
fig.tight_layout()
savefig(dir_save+'modfit_'+suffix+'.eps', format='eps', dpi=1200)

/home/hp/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [375]:
Blos, Btrans = mo_plot[0][50,8], sqrt(mo_plot[0][50,9]**2+mo_plot[0][50,10]**2)
previewImage(Blos)

# Invert again

In [320]:
nicole_input = dir_nicole + 'NICOLE.input'
INPUT = nic.set_nicole_input_params(Mode = 'I',
                                    Printout__detail = 1,
                                    Input__model = model_fit,
                                    Observed__profiles = profile_obs,
                                    Output__profiles = profile_fit2,
                                    Heliocentric__angle = 0.96, 
                                    Output__model = model_fit2,
                                    noise__level = 0.003,
                                    Maximum__number__of__inversions=5,
                                    Continuum__reference = 3,
                                    Debug__mode = 1)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Region 1',
                                     First__wavelength = wav_ran[0,0], 
                                     Wavelength__step = lin_disp,
                                     Number__of__wavelengths = wav_n)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Line 1',
                                     Line = 'CaII__8542',
                                     Mode = 'NLTE')
INPUT = nic.set_nicole_regions_lines(INPUT, 'Nodes',
                                     Temperature = 7,
                                     Velocity = 3,
                                     Microturbulence = 1,
                                     Bz = 3,
                                     By = 3,
                                     Bx = 3)
# INPUT = nic.set_nicole_regions_lines(INPUT, 'NLTE',
#                                      Linear__formal__solution = 1)

F = open(nicole_input, 'w')
F.write(INPUT)
F.close()

# Plot again

In [322]:
IND = 0
s_out2 = read_native_profile([profile_fit2])[0]
# s_out2 = [gaussian_filter1d(s_out[0], 3, axis=1)]
fig = figure(None, figsize=(10,6))
nic.plot_profile(fig, [stokes_temp[:,:,IND,0]], color='k', ls='-.', lw=0.5)
nic.plot_profile(fig, [s_out2[:,:,0,IND]], color='k', ls='--', lw=1)
fig.tight_layout()
savefig(dir_save+'profit2_'+suffix+'.eps', format='eps', dpi=1200)

/home/hp/Dropbox/Inversions/NICOLE.git/run/ZIMPOL3/profit2_CaII_9_74_1_4.pro 
 b'nicole2.3bp     '
1 1 600


/home/hp/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [325]:
mo_pars2, mo_head2, mo_plot2, mo_const2, mo_abu2 = read_native_model(model_fit2)
fig = figure(None, figsize=(10,6))
nic.plot_model_bvtp(fig, [mi_plot], True, color='k', ls=':')
nic.plot_model_bvtp(fig, [mo_plot2[0][:,:,0,IND]], True, color='k', ls = '--')
fig.tight_layout()
savefig(dir_save+'modfit2_'+suffix+'.eps', format='eps', dpi=1200)

/home/hp/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [326]:
Blos, Btrans = mo_plot2[0][50,8], sqrt(mo_plot2[0][50,9]**2+mo_plot2[0][50,10]**2)
previewImage(Blos)

In [ ]:
# Photosphere -10
# Chromosphere 50